# Final Project DL model Notebook - sinple CNN model with two convolutional layers - v01

#### imports, settings and constants

run log: 

- local
- sample data set
- batch_size = 16
- epochs = 2
- learning_rate = 1e-4

In [ ]:
import sys

from keras.models import Sequential
from keras.layers import BatchNormalization, Flatten, Dense, Conv2D, MaxPooling2D
from keras.optimizers import Adam
import numpy as np

from pathlib import Path

sys.path.append(str(Path.cwd().parent))
from utils import *

current_dir = Path.cwd()
HOME_DIR = current_dir.parent
DATA_DIR = HOME_DIR.joinpath('data')

# comment out one of the two path options to toggle between sample directory and all data
# path = DATA_DIR
path = DATA_DIR.joinpath('sample')
train_path = path.joinpath('train')
val_path = path.joinpath('valid')
test_path = path.joinpath('test')
results_path = path.joinpath('tesults')

# training variables
batch_size = 16
epochs = 2
learning_rate = 1e-3

#### getting training and validation data in batches

In [ ]:
batches = get_in_batches(train_path, batch_size=batch_size)
val_batches = get_in_batches(val_path, batch_size=batch_size)
test_batches = get_in_batches(test_path, batch_size=batch_size)

#### and getting the classes, labels and filenames for each batch

In [ ]:
trn_classes = batches.classes
val_classes = val_batches.classes
trn_labels = onehot(batches.classes)
val_labels = onehot(val_batches.classes)
trn_filenames = batches.filenames
val_filenames = val_batches.filenames

#### defining CNN model

In [ ]:
model = Sequential([
            BatchNormalization(axis=1, input_shape=(3, 224, 224)),
            Conv2D(32, 3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Conv2D(64, 3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3, 3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
    ])
model.compile(Adam(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

#### fit_generator() consuming the batches provided by the ImageDataGenerator to fit the model to the data

batches.n = samples in the dataset

batches.batch_size = number of samples in a batch

using batches.n // batches.batch_size ensures we go over the entire dataset once per epoch

validation_steps = number of batches of samples to validate against

In [ ]:
history = model.fit_generator(batches,
                    steps_per_epoch=batches.n//batches.batch_size,
                    validation_data=val_batches,
                    validation_steps=1,
                    epochs=epochs,
                    callbacks=[reduce_lr, stop_early])

In [ ]:
plot_acc_and_loss(history)

#### test performance and plot confusion matrix on one batch of 200 images

In [ ]:
test_set = get_in_batches(val_path, shuffle=False, class_mode=None, batch_size=200)
pred_classes = model.predict_generator(test_set, 1)
pred_classes = np.argmax(pred_classes, axis=1)
act_classes = test_set.classes

In [ ]:
cm = confusion_matrix(act_classes, pred_classes)
plot_confusion_matrix(cm, val_batches.class_indices)
plt.figure()
plt.show()

In [ ]:
model.optimizer.lr = 1e-4
history = model.fit_generator(batches,
                    steps_per_epoch=batches.batch_size,
                    validation_data=val_batches,
                    validation_steps=val_batches.batch_size,
                    epochs=epochs)

In [ ]:
plot_acc_and_loss(history)

In [ ]:
test_set = get_in_batches(val_path, shuffle=False, class_mode=None, batch_size=200)
pred_classes = model.predict_generator(test_set, 1)
pred_classes = np.argmax(pred_classes, axis=1)
act_classes = test_set.classes

In [ ]:
cm = confusion_matrix(act_classes, pred_classes)
plot_confusion_matrix(cm, val_batches.class_indices)
plt.figure()
plt.show()

In [ ]:
pred_classes

In [ ]:
act_classes

#### validating the model performance on the val set

Running the evaluate generator returns the cost and accuracy of the model. Doing it in a loop allows us to confirm that the performance is stable. Results should be very similar for all runs. This takes _very_ long though

In [ ]:
rnd_batches = get_in_batches(val_path, batch_size=batch_size, shuffle=True)
val_res = [model.evaluate_generator(rnd_batches, rnd_batches.samples) for i in range(3)]
np.round(val_res, 3
print(val_res)